In [1]:
import pandas as pd
import numpy as np
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, SimpleRNN, Dense, Dropout, Flatten, Bidirectional,TimeDistributed
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Load training data
# Training cycle index
cyc = range(1,46)

# cyc = [1, 9, 18, 27, 36, 45]

# Training current profile
arr = ['1C','2C','4C','6C','UDDS','US06']#,'3C','5C','SC04','LA92']

# arr = ['1C','2C','4C','6C','UDDS','US06','3C','5C','SC04','LA92']

j=1
for x in cyc:
    x = "%.2d" % x
    for y in arr:
        if j == 1:
            name1 = 'Journal_data/Exp_SOH/sam2/cycle_' + str(x) + '/Cycle_' + str(x) + '_Real_'+ str(y) + '_discharge_cut.csv'
            Real = np.array(pd.read_csv(name1,header = None))
            name2 = 'Journal_data/Exp_SOH/sam2/cycle_' + str(x) + '/Cycle_' + str(x) + '_NDC_'+ str(y) + '_discharge.csv'
            NDC = np.array(pd.read_csv(name2,header = None))
            NDC = np.concatenate((NDC,NDC[0,1]*np.ones((len(NDC),1))), axis = 1) # Add SOH value
        
        if j > 1:
            name1 = 'Journal_data/Exp_SOH/sam2/cycle_' + str(x) + '/Cycle_' + str(x) + '_Real_'+ str(y) + '_discharge_cut.csv'
            Real = np.concatenate((Real, np.array(pd.read_csv(name1,header = None))),axis = 0)
            name2 = 'Journal_data/Exp_SOH/sam2/cycle_' + str(x) + '/Cycle_' + str(x) + '_NDC_'+ str(y) + '_discharge.csv'
            temp = np.array(pd.read_csv(name2,header = None))
            temp = np.concatenate((temp,temp[0,1]*np.ones((len(temp),1))), axis = 1) # Add SOH value
            NDC = np.concatenate((NDC,temp),axis = 0)
        j=j+1

In [3]:
print("Concatenate SPM train length: " + str(len(NDC)))
print("Concatenate Real train length: " + str(len(Real)))

Concatenate SPM train length: 581567
Concatenate Real train length: 581567


In [4]:
## Learning deltaV
deltaV = Real.reshape(len(Real),1) - NDC[:,0].reshape(len(NDC),1)

In [5]:
#Construct training data
trainX = np.zeros((len(NDC),6))
# trainX[:,0] = SPM[:,0] # 0: Voltage
trainX[:,0] = NDC[:,1] # 1: Bulk SoC
trainX[:,1] = NDC[:,2] # 2: Surface SoC
trainX[:,2] = NDC[:,3] # 3: V1
# trainX[:,3] = NDC[:,4] # 4: SoC
# trainX[:,4] = NDC[:,5] # 5: R0
trainX[:,3] = -NDC[:,6] # 6: current
trainX[:,4] = NDC[:,7] # 7: temperature
trainX[:,5] = NDC[:,8] # 8: SoH

# trainY = Real
trainY = deltaV

In [6]:
scaler1 = MinMaxScaler()
scaler1.fit(trainX)
normalizedX = scaler1.transform(trainX)
scaler2 = MinMaxScaler()
scaler2.fit(trainY)
normalizedY = scaler2.transform(trainY)

In [7]:
trainX = normalizedX
trainY = normalizedY

In [8]:
initScore = math.sqrt(mean_squared_error(Real.reshape(len(Real),1), NDC[:,0].reshape(len(NDC),1)))
print("Initial Score: RMSE " + str(initScore))

Initial Score: RMSE 0.08768715665209224


In [9]:
model = Sequential()

model.add(Dense(32, input_dim=6, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1,activation='linear'))

model.compile(loss='mean_squared_error',optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                224       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,313
Trainable params: 1,313
Non-trainable params: 0
_________________________________________________________________


In [10]:
# train NN
model.fit(trainX, trainY, epochs=300, verbose=0)

In [11]:
trainPredict = model.predict(trainX)
trainPredict = scaler2.inverse_transform(trainPredict.reshape(len(trainPredict),1))
trainScore = math.sqrt(mean_squared_error(deltaV, trainPredict.reshape(len(trainPredict),1)))
print("Train Score: RMSE " + str(trainScore))

Train Score: RMSE 0.009706787003386091


In [ ]:
# # Testing cycle index
# x = "%.2d" %28
# # Testing current profile
# y = 'SC04'

# name1 = 'Journal_data/Exp_SOH/cycle_' + str(x) + '/Cycle_' + str(x) + '_Real_'+ str(y) + '_discharge_cut.csv'
# Real_T = np.array(pd.read_csv(name1,header = None))
# name2 = 'Journal_data/Exp_SOH/cycle_' + str(x) + '/Cycle_' + str(x) + '_NDC_'+ str(y) + '_discharge.csv'
# NDC_T = np.array(pd.read_csv(name2,header = None))
# NDC_T = np.concatenate((NDC_T,NDC_T[0,1]*np.ones((len(NDC_T),1))), axis = 1) # Add SOH value

# #Construct testing data
# testX = np.zeros((len(NDC_T),6))
# # testX[:,0] = NDC_T[:,0]
# testX[:,0] = NDC_T[:,1]
# testX[:,1] = NDC_T[:,2]
# testX[:,2] = NDC_T[:,3]
# # testX[:,3] = NDC_T[:,4]
# # testX[:,4] = NDC_T[:,5]
# testX[:,3] = -NDC_T[:,6]
# testX[:,4] = NDC_T[:,7]
# testX[:,5] = NDC_T[:,8]

# testY =  Real_T.reshape(len(Real_T),1) - NDC_T[:,0].reshape(len(NDC_T),1)
# # testY = Real_T
# normalizedXtest = scaler1.transform(testX)

In [ ]:
# testPredict = model.predict(normalizedXtest)
# testPredict = scaler2.inverse_transform(testPredict)
# testScore = math.sqrt(mean_squared_error(testY,testPredict.reshape(len(testPredict),1)))
# print("Test Score: RMSE " + str(testScore))
# InitialScore = math.sqrt(mean_squared_error(Real_T, NDC_T[:,0]))
# print("Initial Score: RMSE " + str(InitialScore))

In [ ]:
# # save data voltage residual
# np.savetxt('Results(Journal paper)/HYBRID1/predict_H1_Test_3C.csv', NDC_T[:,0].reshape(len(NDC_T),1) + testPredict.reshape(len(testPredict),1), delimiter=',')

In [ ]:
# # save data termainal voltage
# np.savetxt('Results(Journal paper)/HYBRID2/predict_H2_Test_UDDS.csv', testPredict.reshape(len(testPredict),1), delimiter=',')

In [21]:
# Testing cycle index
Score = []
for idx in range(1,46):
    
    x = "%.2d" %idx
    # Testing current profile
    y = '6C'

    name1 = 'Journal_data/Exp_SOH/sam3/cycle_' + str(x) + '/Cycle_' + str(x) + '_Real_'+ str(y) + '_discharge_cut.csv'
    Real_T = np.array(pd.read_csv(name1,header = None))
    name2 = 'Journal_data/Exp_SOH/sam3/cycle_' + str(x) + '/Cycle_' + str(x) + '_NDC_'+ str(y) + '_discharge.csv'
    NDC_T = np.array(pd.read_csv(name2,header = None))
    NDC_T = np.concatenate((NDC_T,NDC_T[0,1]*np.ones((len(NDC_T),1))), axis = 1) # Add SOH value

    #Construct testing data
    testX = np.zeros((len(NDC_T),6))
    # testX[:,0] = NDC_T[:,0]
    testX[:,0] = NDC_T[:,1]
    testX[:,1] = NDC_T[:,2]
    testX[:,2] = NDC_T[:,3]
    # testX[:,3] = NDC_T[:,4]
    # testX[:,4] = NDC_T[:,5]
    testX[:,3] = -NDC_T[:,6]
    testX[:,4] = NDC_T[:,7]
    testX[:,5] = NDC_T[:,8]

    testY =  Real_T.reshape(len(Real_T),1) - NDC_T[:,0].reshape(len(NDC_T),1)
    # testY = Real_T
    normalizedXtest = scaler1.transform(testX)

    testPredict = model.predict(normalizedXtest)
    testPredict = scaler2.inverse_transform(testPredict)
    testScore = math.sqrt(mean_squared_error(testY,testPredict.reshape(len(testPredict),1)))
#     print("Test Score: RMSE " + str(testScore))
    InitialScore = math.sqrt(mean_squared_error(Real_T, NDC_T[:,0]))
#     print("Initial Score: RMSE " + str(InitialScore))
    
    Score.append([InitialScore*1000, testScore*1000])
    
# save Score 
np.savetxt('Sam3_Testscore_' + y + '.csv', Score, delimiter=',')